<a href="https://colab.research.google.com/github/aditichak22/ml-logistic-regression/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1: https://drive.google.com/file/d/19SBv8qSbSA2IvPjmyEEPRED0giBTj9kb/view?usp=sharing


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

def prepareData(datafile):
  # data = pd.read_csv(datafile, index_col=False, names=["y", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20", "x21"])
  data = pd.read_csv(datafile, header = None).values
  data[:,0][data[:,0] == 0] = -1
  X = data[:, 1:22]
  y = data[:, 0:1]
  
  # m = no. of training samples, n = no. of features
  m,n = X.shape
  y = y.reshape(m,1)

  return data, X, y

# data, X_train, Y_train = prepareData("/content/drive/MyDrive/park_train.data")
# Y_train

In [ ]:
import numpy as np
import pandas as pd

data, X_train, Y_train = prepareData("/content/drive/MyDrive/park_train.data")

init_parameters = {} 
init_parameters["weight"] = np.zeros(X_train.shape[1])
init_parameters["bias"] = 0

def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

def optimize(x, y, learning_rate,iterations,parameters): 

    size = x.shape[0]
    weight = parameters["weight"] 
    bias = parameters["bias"]
    for i in range(iterations): 
            sigma = sigmoid(np.dot(x, weight) + bias)
            loss = -1/size * np.sum(y * np.log(sigma)) + (1 - y) * np.log(1-sigma)
            dW = 1/size * np.dot(x.T, (sigma - y))
            db = 1/size * np.sum(sigma - y)
            weight -= learning_rate * dW
            bias -= learning_rate * db 
        
    parameters["weight"] = weight
    parameters["bias"] = bias
    return parameters    

def train(x, y, learning_rate,iterations):
    parameters_out = optimize(x, y, learning_rate, iterations ,init_parameters)
    return parameters_out

parameters_out = train(X_train, Y_train, learning_rate = 0.02, iterations = 500)
parameters_out



In [ ]:
import pandas as pd
import numpy as np
import sys
import scipy
from scipy.special import expit, logit

def normalize(X):    
    M,N = X.shape    
    mean_ = X.mean(axis =0)
    std_ = np.std(X)
    X_scaled = (X-mean_)/std_   
    return mean_, std_, X_scaled

def probabilities(W, bias, X):
    # print("fsdfs")
    # print(W)
    functional_margin = np.dot(X,W) + bias
    # print("here")
    # print(np.dot(X,W))
    e =  scipy.special.expit(functional_margin)
    P_x_y_1 = (e/(1+e))
    P_x_y_m1 = (1/(1+e))
    return P_x_y_1, P_x_y_m1, functional_margin

def gradients(W, X, bias, Y):
    M,N = X.shape
    P_x_y_1, P_x_y_m1, functional_margin = probabilities(W, bias, X)
    G_b = ((0.5*(Y+1)) - P_x_y_1).sum()
    # print("DW!!",-P_x_y_1)
    G_w = np.sum(X*((0.5*(Y+1))-P_x_y_1), axis=0).reshape(N,1)
    likelihood = np.sum( (0.5*(Y+1)*functional_margin) - np.log10(1 + scipy.special.expit(functional_margin)), axis=0)
    # print("likelihood", likelihood)
    return (G_b, G_w, likelihood)

def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output  

def logistic_train(X, Y, alpha):
    M, N = X.shape
    W = np.ones(N).reshape(N,1)*(0.25)
    bias = 0.75
    iterations = 0
    print('PRINTING ACCURACY AFTER EACH ITERATION ')
    G_b, G_w, likelihood = gradients(W, X, bias, Y)
    current_likelihood = previous_likelihood = likelihood
    max_iterations = 500
    while iterations <  max_iterations: 
        # step = 2.0/(2.0 + iterations)
        step = alpha
        previous_likelihood = current_likelihood
        W = W + step * G_w
        bias = bias + step * G_b
        iterations = iterations + 1
        (G_b, G_w, current_likelihood) = gradients(W, X, bias, Y)
        # print(iterations, G_b, accuracy(X, Y, W, bias), (current_likelihood - previous_likelihood), current_likelihood, previous_likelihood)
        # if iterations%10000 == 0:
        #     print(iterations, G_b, accuracy(X, Y, W, bias), (current_likelihood - previous_likelihood), current_likelihood, previous_likelihood)

        if abs(current_likelihood-previous_likelihood) < 1e-7:
            break
        # mj = mj - 1
        #print(G_b, accuracy(X, Y, W, bias), current_likelihood-previous_likelihood, current_likelihood, previous_likelihood)
        
    return (W, bias, iterations)

def accuracy(X, Y, W, bias):
    M,N = X.shape
    Y_pred = np.dot(X, W) + bias
    Y_pred[Y_pred>=0] = 1
    Y_pred[Y_pred<0] = -1
    misclassifications =  np.sum(abs(Y-Y_pred))/2
    return (1- misclassifications/M)*100

def logistic_validate(X_train, Y_train, X_val, Y_val):
    best_W = best_bias = best_alpha = None
    best_accuracy = 0
    for i in range(-5, 0):
        print('------------------------------------------------------------')
        alpha = 10**i
        print('VALIDATION FOR alpha ', alpha)
        (W, bias, iterations) = logistic_train(X_train, Y_train, alpha)
        training_accuracy = accuracy(X_train, Y_train, W, bias)  
        current_accuracy = accuracy(X_val, Y_val, W, bias)
        test_accuracy = accuracy(X_test, Y_test, W, bias)
        print('------------------------------------------------------------')
        print('------------------------------------------------------------')
        print('TRAINING ACCURACY ', training_accuracy)
        print('VALIDATION ACCURACY ', current_accuracy)            
        print('TEST ACCURACY ', test_accuracy)
        print('------------------------------------------------------------')
        print('------------------------------------------------------------')
        if current_accuracy > best_accuracy :
            best_accuracy = current_accuracy
            best_W = W
            best_bias = bias     
            best_alpha = alpha
        
        print('------------------------------------------------------------')
        
    return (best_W, best_bias, best_alpha)

# (X_train, Y_train) = prepare_data(data_train)
data, X_train, Y_train = prepareData("/content/drive/MyDrive/park_train.data")
# X_scaled_train, X_mean, X_sd = normalize(X_train)
# (X_val, Y_val) = prepare_data(data_val)
data, X_val, Y_val = prepareData("/content/drive/MyDrive/park_validation.data")
# #X_val = rescale(X_val, X_mean, X_sd)
data, X_test, Y_test = prepareData("/content/drive/MyDrive/park_test.data")
# #X_test = rescale(X_test, X_mean, X_sd)

W, bias, alpha = logistic_validate(X_train, Y_train, X_val, Y_val)
training_accuracy = accuracy(X_train, Y_train, W, bias)
print('ACCURACY ON TRAINING SET ', training_accuracy)
test_accuracy = accuracy(X_test, Y_test, W, bias)
print('ACCURACY ON TEST SET ', test_accuracy)

------------------------------------------------------------
VALIDATION FOR alpha  1e-05
PRINTING ACCURACY AFTER EACH ITERATION 
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
VALIDATION FOR alpha  0.0001
PRINTING ACCURACY AFTER EACH ITERATION 
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
-------------------------------------

If the input data is prfectly linearly separable, the logistic regression model can no longer be trained because the weight would not converge as optimal weight would be infinite. 

This problem can be solved by introducing penalization of weights or defining a prior probability distribution of weights. 

In [ ]:
def probabilities(W, bias, X):
    # print("fsdfs")
    # print(W)
    functional_margin = np.dot(X,W) + bias
    # print("here")
    # print(np.dot(X,W))
    e =  scipy.special.expit(functional_margin)
    P_x_y_1 = (e/(1+e))
    P_x_y_m1 = (1/(1+e))
    return P_x_y_1, P_x_y_m1, functional_margin

def gradients(W, X, bias, Y, lamb):
    M,N = X.shape
    P_x_y_1, P_x_y_m1, functional_margin = probabilities(W, bias, X)
    G_b = ((0.5*(Y+1))-P_x_y_1).sum()
    G_w = np.sum(X*((0.5*(Y+1))-P_x_y_1), axis=0).reshape(N,1) - lamb*W
    likelihood = np.sum( 0.5*(Y+1)*functional_margin - np.log(1 + scipy.special.expit(functional_margin)), axis=0) - (lamb)*np.sum(W**2,axis=0)
    return (G_b, G_w, likelihood)

def calculate_loss(W, Y, X, bias):
    return np.sum((0.5)*(Y+1)*(np.dot(X, W)+bias) - np.log(1 + scipy.special.expit(np.dot(X,W) + bias)))

def logistic_train(X, Y, alpha, lamb):
    M, N = X.shape
    W = np.ones(N).reshape(N,1)*(0.5)
    bias = 0.5
    iterations = 0
    #print('PRINTING ACCURACY AFTER EACH ITERATION ')
    (G_b, G_w, likelihood) = gradients(W, X, bias, Y, lamb)
    current_likelihood = previous_likelihood = likelihood
    max_iterations = 500
    while iterations <  max_iterations:
        previous_likelihood = current_likelihood
        W = W + alpha * G_w
        bias = bias + alpha * G_b
        iterations = iterations + 1
        (G_b, G_w, current_likelihood) = gradients(W, X, bias, Y, lamb)
        # if iterations%1000 == 0:
        #     print(iterations, G_b, accuracy(X, Y, W, bias), current_likelihood-previous_likelihood, current_likelihood, previous_likelihood)
        if abs(current_likelihood-previous_likelihood) < 1e-4:
            break
        #print(G_b, accuracy(X, Y, W, bias), current_likelihood-previous_likelihood)
        
    return (W, bias, iterations)

def accuracy(X, Y, W, bias):
    M,N = X.shape
    Y_pred = np.dot(X, W) + bias
    Y_pred[Y_pred>0] = 1
    Y_pred[Y_pred<0] = -1
    misclassifications =  np.sum(abs(Y-Y_pred))/2
    return (1- misclassifications/M)*100

def logistic_validate(X_train, Y_train, X_val, Y_val, X_test, Y_test):
    best_W = best_bias = best_alpha = best_lamb = None
    best_accuracy = 0
    for i in range(-6, -3):        
        alpha = 10**i
        for j in range(-4,5):
            lamb = 10**j
            print('------------------------------------------------------------')
            print('VALIDATION FOR alpha ', alpha, ', lambda ', lamb)
            (W, bias, iterations) = logistic_train(X_train, Y_train, alpha, lamb)
            training_accuracy = accuracy(X_train, Y_train, W, bias)  
            current_accuracy = accuracy(X_val, Y_val, W, bias)
            test_accuracy = accuracy(X_test, Y_test, W, bias)
            print('------------------------------------------------------------')
            print('------------------------------------------------------------')
            print('TRAINING ACCURACY ', training_accuracy)
            print('VALIDATION ACCURACY ', current_accuracy)            
            print('TEST ACCURACY ', test_accuracy)
            print('------------------------------------------------------------')
            print('------------------------------------------------------------')
            if current_accuracy > best_accuracy :
                best_accuracy = current_accuracy
                best_W = W
                best_bias = bias     
                best_alpha = alpha
                best_lamb = lamb
            print('------------------------------------------------------------')
        
        
    return (best_W, best_bias, best_alpha, best_lamb)

data, X_train, Y_train = prepareData("/content/drive/MyDrive/park_train.data")
data, X_val, Y_val = prepareData("/content/drive/MyDrive/park_validation.data")
data, X_test, Y_test = prepareData("/content/drive/MyDrive/park_test.data")

(W, bias, alpha, lamb) = logistic_validate(X_train, Y_train, X_val, Y_val, X_test, Y_test)
training_accuracy = accuracy(X_train, Y_train, W, bias)
print("Weights", W)
print('ACCURACY ON TRAINING SET ', training_accuracy)
test_accuracy = accuracy(X_test, Y_test, W, bias)
print('ACCURACY ON TEST SET ', test_accuracy)

------------------------------------------------------------
VALIDATION FOR alpha  1e-06 , lambda  0.0001
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
VALIDATION FOR alpha  1e-06 , lambda  0.001
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
------------------------------------------------------------
------------------------

In [ ]:
def probabilities(W, bias, X):
    functional_margin = np.dot(X,W) + bias
    e = scipy.special.expit(functional_margin)
    P_x_y_1 = (e/(1+e))
    P_x_y_m1 = (1/(1+e))
    return (P_x_y_1, P_x_y_m1, functional_margin)

def gradients(W, X, bias, Y, lamb):
    M,N = X.shape
    P_x_y_1, P_x_y_m1, functional_margin = probabilities(W, bias, X)
    G_b = ((0.5*(Y+1))-P_x_y_1).sum()
    norm_derivative = W
    norm_derivative[norm_derivative<0] = -1
    norm_derivative[norm_derivative>=0] = 1
    G_w = np.sum(X*((0.5*(Y+1))-P_x_y_1), axis=0).reshape(N,1) - lamb*norm_derivative
    likelihood = np.sum( 0.5*(Y+1)*functional_margin - np.log(1 + scipy.special.expit(functional_margin)), axis=0) - lamb*np.sum(abs(W), axis=0)
    return G_b, G_w, likelihood

def logistic_train(X, Y, alpha, lamb):
    M, N = X.shape
    W = np.ones(N).reshape(N,1)*(0.25)
    bias = 0.75
    iterations = 0
    print('PRINTING ACCURACY AFTER EACH ITERATION ')
    (G_b, G_w, likelihood) = gradients(W, X, bias, Y, lamb)
    current_likelihood = previous_likelihood = likelihood
    max_iterations = 500
    while iterations <  max_iterations:
        previous_likelihood = current_likelihood
        W = W + alpha * G_w
        bias = bias + alpha * G_b
        iterations = iterations + 1
        (G_b, G_w, current_likelihood) = gradients(W, X, bias, Y, lamb)
        if iterations%10000 == 0:
            print(iterations, G_b, accuracy(X, Y, W, bias), current_likelihood-previous_likelihood, current_likelihood, previous_likelihood)
        
        if abs(current_likelihood-previous_likelihood) < 1e-7:
            break
        
        #print(G_b, accuracy(X, Y, W, bias), current_likelihood-previous_likelihood, current_likelihood, previous_likelihood)
        
    return (W, bias, iterations)

def accuracy(X, Y, W, bias):
    M,N = X.shape
    Y_pred = np.dot(X, W) + bias
    Y_pred[Y_pred>0] = 1
    Y_pred[Y_pred<0] = -1
    misclassifications =  np.sum(abs(Y-Y_pred))/2
    return (1- misclassifications/M)*100

def logistic_validate(X_train, Y_train, X_val, Y_val):
    best_W = best_bias = best_alpha = best_lamb = None
    best_accuracy = 0
    for i in range(-5, -4):
        print('------------------------------------------------------------')
        alpha = 10**i
        for j in range(-6,5):
            lamb = 10**j
            print('VALIDATION FOR alpha ', alpha, ', lambda ', lamb)
            (W, bias, iterations) = logistic_train(X_train, Y_train, alpha, lamb)
            training_accuracy = accuracy(X_train, Y_train, W, bias)  
            current_accuracy = accuracy(X_val, Y_val, W, bias)
            test_accuracy = accuracy(X_test, Y_test, W, bias)
            print('------------------------------------------------------------')
            print('------------------------------------------------------------')
            print('TRAINING ACCURACY ', training_accuracy)
            print('VALIDATION ACCURACY ', current_accuracy)            
            print('TEST ACCURACY ', test_accuracy)
            print('------------------------------------------------------------')
            print('------------------------------------------------------------')
            if current_accuracy > best_accuracy :
                best_accuracy = current_accuracy
                best_W = W
                best_bias = bias     
                best_alpha = alpha
                best_lamb = lamb
        
        print('------------------------------------------------------------')
        
    return (best_W, best_bias, best_alpha, best_lamb)


data, X_train, Y_train = prepareData("/content/drive/MyDrive/park_train.data")
data, X_val, Y_val = prepareData("/content/drive/MyDrive/park_validation.data")
data, X_test, Y_test = prepareData("/content/drive/MyDrive/park_test.data")

(W, bias, alpha, lamb) = logistic_validate(X_train, Y_train, X_val, Y_val)
print("Weights", W)
training_accuracy = accuracy(X_train, Y_train, W, bias)
print('ACCURACY ON TRAINING SET ', training_accuracy)
val_accuracy = accuracy(X_val, Y_val, W, bias)
print('ACCURACY ON VALIDATION SET ', val_accuracy)
test_accuracy = accuracy(X_test, Y_test, W, bias)
print('ACCURACY ON TEST SET ', test_accuracy)


------------------------------------------------------------
VALIDATION FOR alpha  1e-05 , lambda  1e-06
PRINTING ACCURACY AFTER EACH ITERATION 
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
------------------------------------------------------------
VALIDATION FOR alpha  1e-05 , lambda  1e-05
PRINTING ACCURACY AFTER EACH ITERATION 
------------------------------------------------------------
------------------------------------------------------------
TRAINING ACCURACY  78.2051282051282
VALIDATION ACCURACY  74.13793103448276
TEST ACCURACY  72.88135593220339
------------------------------------------------------------
------------------------------------------------------------
VALIDATION FOR alpha  1e-05 , lambda  0.0001
PRINTING ACCURACY AFTE

As the weights get smaller, L1 updates the weights by the same lambda but L2 updates by Lambda * W. A net change in W increases L1 rather than L2 as the weights get smaller. So L produces sparser weights than L2. 